In [ ]:
from selenium import webdriver 
from selenium.webdriver.chrome.options import Options
from fake_useragent import UserAgent
from selenium.webdriver import ActionChains 
from time import sleep
import time

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

In [ ]:
options = Options()
ua = UserAgent()
userAgent = ua.random
options.add_argument('--start-fullscreen')  # 전체화면
options.add_argument(f'user-agent={userAgent}') # useragent값
options.add_argument('--disable-blink-features=AutomationControlled')   # 봇 피하기
driver = webdriver.Chrome(chrome_options=options, executable_path=r'./chromedriver')    # 크롬주소
sleep(3)

In [ ]:
driver.get("https://www.nbkorea.com/product/productList.action?cateGrpCode=250110&cIdx=1280")
sleep(3)


In [ ]:
# 신발 더보기 클릭
more_btn=driver.find_element_by_class_name('btn_area')
count=int(driver.find_element_by_id('moreList').get_attribute('existcnt'))
for i in range(count//30+1):
    more_btn.click()
    sleep(1)

In [ ]:
product=driver.find_element_by_id('prodList')
items=product.find_elements_by_tag_name('li')

# 상품의 data-style, data-cidx속성값을 저장
style_list=[]
for i in items:
    a=i.find_element_by_class_name('pro_area').get_attribute('data-style')
    b=i.find_element_by_class_name('pro_area').get_attribute('data-cidx')
    style_list.append(a+b)

# data-style값을 통해 신발 링크를 들어감.

In [ ]:
len(style_list)

In [ ]:
# 신발 상품 stylecode로 중복제거
style_dict={}

for i in style_list:
    style_code=i[:9]
    color_code=i[9:]
    style_dict[style_code]=color_code

In [ ]:
# 중복 제거 후 신발 url 저장
item_url=[]
for k,v in style_dict.items():
    styleCode=k+v[0]
    cat_num=v[1:]
    item_url.append(f'https://www.nbkorea.com/product/productDetail.action?styleCode={styleCode}&cIdx={cat_num}')

len(item_url)

In [ ]:
result_list=[]

# 남성 신발 카테고리 번호
cat_dict={1287:'라이프스타일',
            1283:'MADE IN USA/UK',
            1288:'러닝',
            1377:'워킹',
            1292:'샌들/슬라이드',
            1377:'워킹',
            1499:'농구',
            1289:'축구',
            1290:'야구/테니스',
            1291:'스케이트보딩'}

for url in item_url:
    # 신발 카테고리
    shoe_cat=cat_dict[int(url.split('cIdx=')[1])]

    driver.get(url)
    sleep(2)

    # 신발 이름
    shoe_name=driver.find_element_by_xpath('//*[@id="displayName"]').text
    # 신발 가격
    shoe_price=driver.find_element_by_id('dispPrice').text
    # 리뷰 수
    review_cnt=driver.find_element_by_xpath('/html/body/div[1]/div[3]/div[1]/div[2]/div[1]/div[2]/div[1]/a').text
    review_cnt=review_cnt.split('(')[1].split(')')[0]

    if int(review_cnt)>0:
        driver.find_element_by_css_selector('#pr_review').click()
        # 리뷰 더보기 클릭
        try:
            review_more=driver.find_element_by_id("reviewMoreList")
            for i in range(int(review_more.get_attribute('data-exist-cnt'))):
                review_more.click()
                sleep(1)
        except:
            pass

        review_list=driver.find_elements_by_class_name('review_detail')
        
        for review in review_list:
            # 리뷰 제목
            review_title=review.find_element_by_class_name('tit').text
            # 리뷰 내용
            review_contents=review.find_element_by_class_name('txt').text
            
            result_list.append({
                '신발 이름':shoe_name,
                '신발가격':shoe_price,
                '신발 카테고리':shoe_cat,
                '리뷰 제목':review_title,
                '리뷰 내용':review_contents,
                '신발 url':url
            })

    else:
        result_list.append({
                '신발 이름':shoe_name,
                '신발가격':shoe_price,
                '신발 카테고리':shoe_cat,
                '리뷰 제목':'리뷰 없음',
                '리뷰 내용':'리뷰 없음',
                '신발 url':url
            })




In [ ]:
df=pd.DataFrame(result_list)
df.to_csv('NewBalance.csv')

In [ ]:
df.shape